In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animals import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "MyNewPassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

logo_png = 'assets/logo.png'
logo_base64 = base64.b64encode(open(logo_png, 'rb').read()).decode('ascii')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.A(
        href="https://www.snhu.edu",
        children = [
            html.Img(
                alt = "Grazioso Salvare Logo",
                src = 'data:image/png;base64,{}'.format(logo_base64),
            )
        ]
    )),
    html.Center("Bloodhound - The Graziano Salvare Animal Search Portal"),
    html.H2("Mike Ross - SNHU"),
    html.Hr(),
    dcc.Dropdown(options = [{'label': 'No Filter', 'value': 'NF'},
                            {'label': 'Water Rescue', 'value': 'WR'}, 
                            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'}, 
                            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DRIT'}], 
                 value = 'NF', 
                 id = 'quick-filter'),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        sort_action='native',
        filter_action='native',
        page_action='native',
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', children = [
        html.Div(className='sixColumns', children = [
            dcc.Graph(figure = {}, id = 'pie-chart-id')
        ]),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
'''
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
'''

@app.callback(
        Output('datatable-id', 'data'),
        [Input('quick-filter', 'value')]
)
def quick_filter(dropdown_value):
    #print("callback executed")
    if dropdown_value == "WR":
    
        filtered = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                 "breed":{"$in":["Labrador Retriever Mix",
                                                                    "Chesapeake Bay Retriever",
                                                                    "Newfoundland"
                                                                ]
                                                         }, 
                                                "sex_upon_outcome":"Intact Female",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$lte":156}
                                                    }
                                               
                               ))
        return filtered.to_dict('records')
        
    elif dropdown_value == "MWR":
        filtered = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                 "breed":{"$in":["German Shepherd",
                                                                    "Alaskan Malamute",
                                                                    "Old English Sheepdog",
                                                                    "Siberian Husky",
                                                                    "Rottweiler"
                                                                ]
                                                         }, 
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$lte":156}
                                                    }
                                               
                               ))
        return filtered.to_dict('records')
        
    elif dropdown_value == "DRIT":
        filtered = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                 "breed":{"$in":["Doberman Pinscher", 
                                                                 "German Shepherd", 
                                                                 "Golden Retriever", 
                                                                 "Bloodhound", 
                                                                 "Rottweiler"
                                                                ]
                                                         }, 
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":20},
                                                "age_upon_outcome_in_weeks":{"$lte":300}
                                                    }
                                               
                               ))
        return filtered.to_dict('records')
        
    elif dropdown_value == "NF":
        return df.to_dict('records')


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    #FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]    

@app.callback(
    Output('pie-chart-id', 'figure'),
    [Input('datatable-id', 'derived_viewport_data')]
)
def update_pie_chart(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, values = dff.value_counts().index, names = dff.value_counts().values)
    return fig
app